# Cambiamenti

- Provare ad usare un solo hidden layer
- Provare a cambiare funzione d'attivazione dell'output layer da lineare a ReLu
- Cambiare l'ultimo grafico


In [1]:
import numpy as np
import pandas as pd
from re import sub
from time import time

from comet_ml import Experiment

from keras.models import Sequential
from keras.layers import Dense, BatchNormalization, Dropout
from keras_tqdm import TQDMNotebookCallback as ktqdm
from keras.utils import normalize
from keras.wrappers.scikit_learn import KerasRegressor
from keras.callbacks import TensorBoard
from keras.optimizers import Adam, SGD
from keras.regularizers import l1, l2
from keras.initializers import RandomUniform, RandomNormal
from keras.layers.advanced_activations import LeakyReLU

from tensorflow.nn import relu, softmax

from sklearn import preprocessing
from sklearn.model_selection import train_test_split, StratifiedKFold, KFold
from sklearn.preprocessing import RobustScaler, MinMaxScaler, StandardScaler, Normalizer
from sklearn.impute import SimpleImputer
from sklearn.datasets import make_regression
from sklearn.metrics import mean_squared_error, r2_score

import scipy.stats as st

import seaborn as sns

import matplotlib.pyplot as plt

Using TensorFlow backend.


In [2]:
experiment = Experiment(api_key="CndJ3YmXyZcxmsV8EccJEuu9C",
                        project_name="NN_Thesis", workspace="paologalligit")

COMET INFO: old comet version (1.0.55) detected. current: 1.0.56 please update your comet lib with command: `pip install --no-cache-dir --upgrade comet_ml`
COMET INFO: Experiment is live on comet.ml https://www.comet.ml/paologalligit/nn-thesis/92022b40c32b42b098e7b5f63d4d2dcf



In [3]:
df = pd.read_csv('fifa19.csv')

In [4]:
df.drop(columns=['Unnamed: 0', 'ID', 'Photo', 'Flag', 'Club Logo', 'Real Face', 'Preferred Foot',
                 'Body Type', 'Jersey Number', 'Joined', 'Loaned From', 'Contract Valid Until'],inplace=True)

## Obiettivo: predire valore dei giocatori

Pre-processing: convertire value, wage e release clause da string a float

In [5]:
curs=["Release Clause", "Value", "Wage"]

for cur in curs:
    def curr2val(x):
        x = str(x).replace('€', '')
        if 'K' in x: x = float(str(x).replace('K', '')) * 1000
        else: x = float(str(x).replace('M', '')) * 1000000
        return x
    df[cur] = df[cur].apply(curr2val)
    

Individuare eventuali outlier nella colonna value

In [6]:
def detect_outlier(data, threshold = 3):
    outliers=[]
    mean = np.mean(data)
    std = np.std(data)
    
    for y in data:
        score= (y - mean) / std 
        if np.abs(score) > threshold:
            outliers.append(y)
    return outliers

In [7]:
min_out = min(detect_outlier(df['Value'], threshold = 2))

df = df[df['Value'] < min_out] 
df = df[df['Value'] > 0]

Conversione in interi per le altre label

In [8]:
cols=["LS", "ST", "RS", "LW", "LF", "CF", "RF", "RW","LAM", "CAM", "RAM", "LM", "LCM", "CM", "RCM", "RM", "LWB", "LDM","CDM", "RDM", "RWB", "LB", "LCB", "CB", "RCB", "RB"]
for col in cols:
    df[col] = df[col].str[:-2]
    df[col] = df[col].astype(float)

In [9]:
df['Height'] = df['Height'].str.replace("'",'.')
df['Height'] = df['Height'].astype(float)

df['Weight'] = df['Weight'].str[:-3]
df['Weight'] = df['Weight'].astype(float)

Calcolo correlazione tra i valori per scegliere colonne significative

In [10]:
df_corr = df.corr()

# fig = plt.figure(figsize=(50,20))
# ax = fig.add_subplot(111)
# cax = ax.matshow(df_corr,cmap='coolwarm', vmin=-1, vmax=1)
# fig.colorbar(cax)

# ticks = np.arange(0,len(df_corr.columns),1)
# ax.set_xticks(ticks)
# ax.set_xticklabels(df_corr.columns)
# plt.xticks(rotation=90)
# ax.set_yticks(ticks)
# ax.set_yticklabels(df_corr.columns)

# plt.show()

In [11]:
labels = []
for label in df_corr:
#     if df_corr['Value'][label] < 0 or df_corr['Value'][label] > 0.5: labels.append(label)
    if df_corr['Value'][label] > 0.55: labels.append(label)
        
df_flt = df[labels]        
df_flt.head()      

,Overall,Potential,Value,Wage,LCM,CM,RCM,Reactions,Release Clause
41,88,88,4000000.0,77000.0,NaN,NaN,NaN,79.0,7400000.0
102,85,85,9000000.0,38000.0,70.0,70.0,70.0,85.0,15300000.0
108,85,85,9000000.0,57000.0,63.0,63.0,63.0,83.0,17100000.0
152,84,84,4200000.0,95000.0,63.0,63.0,63.0,80.0,6900000.0
201,83,83,13000000.0,70000.0,NaN,NaN,NaN,78.0,24700000.0


Mescolo le righe del dataset

In [12]:
df_flt = df_flt.sample(frac=1)

train_slice = int(len(df_flt) * 0.8)

train = df_flt[:train_slice]
test = df_flt[train_slice:]

In [13]:
y_train = train.loc[:, ['Value']]
X_train = train.drop(columns='Value')

y_test = test.loc[:, ['Value']]
X_test = test.drop(columns='Value')

Sostiuisco eventuali valori nan con la media della colonna

In [14]:
imputer = SimpleImputer(missing_values = np.nan, strategy = 'mean')
imputer = imputer.fit(X_train)
X_full = imputer.transform(X_train)

imputer = SimpleImputer(missing_values = np.nan, strategy = 'mean')
imputer = imputer.fit(y_train)
y_full = imputer.transform(y_train)

imputer = SimpleImputer(missing_values = np.nan, strategy = 'mean')
imputer = imputer.fit(X_test)
X_test_full = imputer.transform(X_test)

imputer = SimpleImputer(missing_values = np.nan, strategy = 'mean')
imputer = imputer.fit(y_test)
y_test_full = imputer.transform(y_test)

Scalo i valori, sia per i caratteri che per il target

In [15]:
# scaler = RobustScaler()
# scaler = scaler.fit(X_train)
# X_train = scaler.transform(X_train)

# X_train_scaled = preprocessing.scale(X_train)
scaler = StandardScaler().fit(X_full)
X_scaled = scaler.transform(X_full)

scaler_train = StandardScaler().fit(y_full)
y_scaled = scaler_train.transform(y_full) 
# X_train_scaled, X_test_scaled

scaler = StandardScaler().fit(X_test_full)
X_test_scaled = scaler.transform(X_test_full)

In [16]:
def coeff_determination(y_test, y_pred):
    from keras import backend as K
    SS_res =  K.sum(K.square( y_test-y_pred ))
    SS_tot = K.sum(K.square( y_test - K.mean(y_test) ) )
    return ( 1 - SS_res/(SS_tot + K.epsilon()))

In [17]:
def build_nn(neurons):
    model = Sequential()
    # Adding the input layer
    model.add(Dense(256, input_dim = neurons, activation='relu'))#, kernel_regularizer=l2(0.001)))#, kernel_initializer=RandomNormal(mean=0.0, stddev=0.05, seed=None)))
#     model.add(LeakyReLU(alpha=0.01))
    model.add(Dropout(0.1))
    
    # Adding the first hidden layer
    model.add(Dense(128, activation='relu'))#, kernel_regularizer=l2(0.001)))#, kernel_initializer=RandomNormal(mean=0.0, stddev=0.05, seed=None)))
#     model.add(LeakyReLU(alpha=0.01))
    model.add(Dropout(0.1))
    
    # Adding the second hidden layer
    model.add(Dense(64, activation='relu'))#, kernel_regularizer=l2(0.001)))#, kernel_initializer=RandomNormal(mean=0.0, stddev=0.05, seed=None)))
#     model.add(LeakyReLU(alpha=0.01))
    
    # Adding the output layer
    model.add(Dense(1, activation = 'linear'))
    
    return model

In [18]:
ts_board = TensorBoard(log_dir='value_predictions_v3/{}'.format('kfold_' + str(time())))

In [19]:
def mean_abs_error(prediction, target):
    res = 0
    tot = 0
    for i in range(len(target)):
        if target[i][0] != 0 and not np.isnan(prediction[i][0]):
#             print(prediction[i][0], target[i][0])
            res += abs(prediction[i][0] - target[i][0])
            tot += 1
    print(tot, len(target), len(prediction))
    return round(res / tot, 2)

# Cross-validation
### K-Fold

In [20]:
# kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)
kfold = KFold(n_splits=10, random_state=42, shuffle=False)

In [21]:
scores = []
abs_errors = []

nn = build_nn(X_scaled.shape[1])

opt = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
# opt = SGD(lr=0.01, momentum=0.9)
nn.compile(optimizer = opt, loss = 'mean_absolute_error', metrics = [coeff_determination])

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [22]:
%%time

stage = 0

for train, test in kfold.split(X_scaled):
    
    stage += 1
    
    print('#'*60)
    print('\nSTAGE --> {}\n'.format(stage))
    print('#'*60)
    
    X_train_round, X_test_round, y_train_round, y_test_round = X_scaled[train], X_scaled[test], y_scaled[train], y_scaled[test]
        
    nn.fit(X_train_round, y_train_round, batch_size=125, epochs=200, shuffle=True)
    
    # predict
    res = nn.predict(X_test_round)
    print('R2 of round: ', round(r2_score(y_test_round, res), 4))
    
    # evaluate
    loss, acc = nn.evaluate(X_train_round, y_train_round, verbose=1)
    
    reversed_res = scaler_train.inverse_transform(res)
    reversed_label = scaler_train.inverse_transform(y_test_round)
    abs_errors.append(
        {'ACC': acc * 100, 'MAE': mean_abs_error(reversed_res, reversed_label), 'MAX': max(reversed_label), 'MEAN': round(np.mean(reversed_label), 4)}
    )
    
    print("loss: %.4f, acc: %.4f%%" % (loss, acc*100))
    if acc < 0:
        print('X_test round:\n')
        print(X_test_round)
        print('#'*60)
        print('y_test_round:\n')
        print(y_test_round)
        print('#'*60)
    
    scores.append(acc * 100)

############################################################

STAGE --> 1

############################################################
Instructions for updating:
Use tf.cast instead.
Epoch 1/200
12535/12535 [==============================] - 1s 47us/step - loss: 0.1669 - coeff_determination: 0.8797
Epoch 2/200
12535/12535 [==============================] - 0s 33us/step - loss: 0.1124 - coeff_determination: 0.9320
Epoch 3/200
12535/12535 [==============================] - 0s 34us/step - loss: 0.1043 - coeff_determination: 0.9403
Epoch 4/200
12535/12535 [==============================] - 0s 35us/step - loss: 0.0976 - coeff_determination: 0.9455
Epoch 5/200
12535/12535 [==============================] - 0s 33us/step - loss: 0.0937 - coeff_determination: 0.9515
Epoch 6/200
12535/12535 [==============================] - 0s 34us/step - loss: 0.0938 - coeff_determination: 0.9536
Epoch 7/200
12535/12535 [==============================] - 0s 33us/step - loss: 0.0850 - coeff_determination: 0.96

12535/12535 [==============================] - 0s 35us/step - loss: 0.0593 - coeff_determination: 0.9815
Epoch 68/200
12535/12535 [==============================] - 0s 30us/step - loss: 0.0592 - coeff_determination: 0.9819
Epoch 69/200
12535/12535 [==============================] - 0s 31us/step - loss: 0.0590 - coeff_determination: 0.9827
Epoch 70/200
12535/12535 [==============================] - 0s 32us/step - loss: 0.0593 - coeff_determination: 0.9806
Epoch 71/200
12535/12535 [==============================] - 0s 33us/step - loss: 0.0593 - coeff_determination: 0.9815
Epoch 72/200
12535/12535 [==============================] - 0s 32us/step - loss: 0.0589 - coeff_determination: 0.9818
Epoch 73/200
12535/12535 [==============================] - 0s 32us/step - loss: 0.0587 - coeff_determination: 0.9833
Epoch 74/200
12535/12535 [==============================] - 0s 33us/step - loss: 0.0590 - coeff_determination: 0.9824
Epoch 75/200
12535/12535 [==============================] - 0s 32us/s

12535/12535 [==============================] - 0s 31us/step - loss: 0.0544 - coeff_determination: 0.9846
Epoch 137/200
12535/12535 [==============================] - 0s 34us/step - loss: 0.0554 - coeff_determination: 0.9843
Epoch 138/200
12535/12535 [==============================] - 0s 31us/step - loss: 0.0530 - coeff_determination: 0.9858
Epoch 139/200
12535/12535 [==============================] - 0s 29us/step - loss: 0.0528 - coeff_determination: 0.9862
Epoch 140/200
12535/12535 [==============================] - 0s 32us/step - loss: 0.0546 - coeff_determination: 0.9851
Epoch 141/200
12535/12535 [==============================] - 0s 35us/step - loss: 0.0550 - coeff_determination: 0.9844
Epoch 142/200
12535/12535 [==============================] - 0s 31us/step - loss: 0.0551 - coeff_determination: 0.9842
Epoch 143/200
12535/12535 [==============================] - 0s 31us/step - loss: 0.0546 - coeff_determination: 0.9853
Epoch 144/200
12535/12535 [==============================] - 0

12535/12535 [==============================] - 0s 32us/step - loss: 0.0516 - coeff_determination: 0.9847
Epoch 3/200
12535/12535 [==============================] - 0s 31us/step - loss: 0.0508 - coeff_determination: 0.9861
Epoch 4/200
12535/12535 [==============================] - 0s 32us/step - loss: 0.0506 - coeff_determination: 0.9853
Epoch 5/200
12535/12535 [==============================] - 0s 37us/step - loss: 0.0516 - coeff_determination: 0.9831
Epoch 6/200
12535/12535 [==============================] - 0s 32us/step - loss: 0.0514 - coeff_determination: 0.9843
Epoch 7/200
12535/12535 [==============================] - 0s 33us/step - loss: 0.0505 - coeff_determination: 0.9856
Epoch 8/200
12535/12535 [==============================] - 0s 33us/step - loss: 0.0508 - coeff_determination: 0.9845
Epoch 9/200
12535/12535 [==============================] - 0s 33us/step - loss: 0.0508 - coeff_determination: 0.9860
Epoch 10/200
12535/12535 [==============================] - 0s 32us/step - l

12535/12535 [==============================] - 0s 36us/step - loss: 0.0487 - coeff_determination: 0.9856
Epoch 72/200
12535/12535 [==============================] - 0s 34us/step - loss: 0.0496 - coeff_determination: 0.9857
Epoch 73/200
12535/12535 [==============================] - 1s 45us/step - loss: 0.0482 - coeff_determination: 0.9871
Epoch 74/200
12535/12535 [==============================] - 0s 32us/step - loss: 0.0491 - coeff_determination: 0.9863
Epoch 75/200
12535/12535 [==============================] - 0s 32us/step - loss: 0.0492 - coeff_determination: 0.9866
Epoch 76/200
12535/12535 [==============================] - 0s 31us/step - loss: 0.0485 - coeff_determination: 0.9872
Epoch 77/200
12535/12535 [==============================] - 0s 31us/step - loss: 0.0486 - coeff_determination: 0.9855
Epoch 78/200
12535/12535 [==============================] - 0s 31us/step - loss: 0.0492 - coeff_determination: 0.9860
Epoch 79/200
12535/12535 [==============================] - 0s 36us/s

...

12536/12536 [==============================] - 0s 31us/step - loss: 0.0372 - coeff_determination: 0.9929
Epoch 101/200
12536/12536 [==============================] - 0s 32us/step - loss: 0.0376 - coeff_determination: 0.9928
Epoch 102/200
12536/12536 [==============================] - 0s 32us/step - loss: 0.0375 - coeff_determination: 0.9929
Epoch 103/200
12536/12536 [==============================] - 0s 31us/step - loss: 0.0375 - coeff_determination: 0.9928
Epoch 104/200
12536/12536 [==============================] - 0s 31us/step - loss: 0.0381 - coeff_determination: 0.9923
Epoch 105/200
12536/12536 [==============================] - 0s 30us/step - loss: 0.0385 - coeff_determination: 0.9920
Epoch 106/200
12536/12536 [==============================] - 0s 31us/step - loss: 0.0379 - coeff_determination: 0.9925
Epoch 107/200
12536/12536 [==============================] - 0s 31us/step - loss: 0.0381 - coeff_determination: 0.9923
Epoch 108/200
12536/12536 [==============================] - 0

12536/12536 [==============================] - 0s 32us/step - loss: 0.0379 - coeff_determination: 0.9923
Epoch 169/200
12536/12536 [==============================] - 0s 31us/step - loss: 0.0370 - coeff_determination: 0.9926
Epoch 170/200
12536/12536 [==============================] - 0s 33us/step - loss: 0.0378 - coeff_determination: 0.9924
Epoch 171/200
12536/12536 [==============================] - 0s 32us/step - loss: 0.0378 - coeff_determination: 0.9920
Epoch 172/200
12536/12536 [==============================] - 0s 31us/step - loss: 0.0380 - coeff_determination: 0.9922
Epoch 173/200
12536/12536 [==============================] - 0s 31us/step - loss: 0.0381 - coeff_determination: 0.9919
Epoch 174/200
12536/12536 [==============================] - 0s 31us/step - loss: 0.0378 - coeff_determination: 0.9929
Epoch 175/200
12536/12536 [==============================] - 0s 32us/step - loss: 0.0371 - coeff_determination: 0.9926
Epoch 176/200
12536/12536 [==============================] - 0

12536/12536 [==============================] - 0s 32us/step - loss: 0.0381 - coeff_determination: 0.9924
Epoch 36/200
12536/12536 [==============================] - 0s 33us/step - loss: 0.0375 - coeff_determination: 0.9924
Epoch 37/200
12536/12536 [==============================] - 0s 36us/step - loss: 0.0375 - coeff_determination: 0.9928
Epoch 38/200
12536/12536 [==============================] - 1s 41us/step - loss: 0.0378 - coeff_determination: 0.9925
Epoch 39/200
12536/12536 [==============================] - 0s 38us/step - loss: 0.0373 - coeff_determination: 0.9931
Epoch 40/200
12536/12536 [==============================] - 0s 33us/step - loss: 0.0374 - coeff_determination: 0.9927
Epoch 41/200
12536/12536 [==============================] - 0s 32us/step - loss: 0.0373 - coeff_determination: 0.9927
Epoch 42/200
12536/12536 [==============================] - 0s 30us/step - loss: 0.0378 - coeff_determination: 0.9922
Epoch 43/200
12536/12536 [==============================] - 0s 31us/s

12536/12536 [==============================] - 0s 31us/step - loss: 0.0358 - coeff_determination: 0.9926
Epoch 105/200
12536/12536 [==============================] - 0s 33us/step - loss: 0.0367 - coeff_determination: 0.9929
Epoch 106/200
12536/12536 [==============================] - 1s 42us/step - loss: 0.0376 - coeff_determination: 0.9928
Epoch 107/200
12536/12536 [==============================] - 1s 42us/step - loss: 0.0375 - coeff_determination: 0.9927
Epoch 108/200
12536/12536 [==============================] - 0s 35us/step - loss: 0.0363 - coeff_determination: 0.9932
Epoch 109/200
12536/12536 [==============================] - 0s 33us/step - loss: 0.0375 - coeff_determination: 0.9926
Epoch 110/200
12536/12536 [==============================] - 0s 36us/step - loss: 0.0370 - coeff_determination: 0.9928
Epoch 111/200
12536/12536 [==============================] - 0s 26us/step - loss: 0.0372 - coeff_determination: 0.9928
Epoch 112/200
12536/12536 [==============================] - 0

Epoch 173/200
12536/12536 [==============================] - 0s 24us/step - loss: 0.0367 - coeff_determination: 0.9931
Epoch 174/200
12536/12536 [==============================] - 0s 24us/step - loss: 0.0364 - coeff_determination: 0.9930
Epoch 175/200
12536/12536 [==============================] - 0s 24us/step - loss: 0.0365 - coeff_determination: 0.9931
Epoch 176/200
12536/12536 [==============================] - 0s 24us/step - loss: 0.0373 - coeff_determination: 0.9928
Epoch 177/200
12536/12536 [==============================] - 0s 25us/step - loss: 0.0371 - coeff_determination: 0.9928
Epoch 178/200
12536/12536 [==============================] - 0s 33us/step - loss: 0.0369 - coeff_determination: 0.9929
Epoch 179/200
12536/12536 [==============================] - 0s 23us/step - loss: 0.0377 - coeff_determination: 0.9916
Epoch 180/200
12536/12536 [==============================] - 0s 24us/step - loss: 0.0370 - coeff_determination: 0.9929
Epoch 181/200
12536/12536 [=====================

In [23]:
print("%.2f%% (+/- %.2f%%)" % (np.mean(scores), np.std(scores) * 100))
abs_errors

98.66% (+/- 28.03%)


[{'ACC': 98.46953352181428,
  'MAE': 155021.25,
  'MAX': array([13500000.]),
  'MEAN': 1638187.3654},
 {'ACC': 98.41047991163758,
  'MAE': 176566.81,
  'MAX': array([13500000.]),
  'MEAN': 1791198.8514},
 {'ACC': 98.20997278054111,
  'MAE': 150322.2,
  'MAX': array([13500000.]),
  'MEAN': 1656421.3927},
 {'ACC': 98.38626563144862,
  'MAE': 156036.71,
  'MAX': array([13000000.]),
  'MEAN': 1689310.8399},
 {'ACC': 98.60736586917288,
  'MAE': 142709.4,
  'MAX': array([13500000.]),
  'MEAN': 1648173.0079},
 {'ACC': 98.60622432348877,
  'MAE': 159350.97,
  'MAX': array([13500000.]),
  'MEAN': 1734350.323},
 {'ACC': 98.91889491497781,
  'MAE': 128331.24,
  'MAX': array([13000000.]),
  'MEAN': 1737376.1665},
 {'ACC': 99.033824490702,
  'MAE': 124036.26,
  'MAX': array([13500000.]),
  'MEAN': 1736302.9433},
 {'ACC': 99.04657334526709,
  'MAE': 110208.46,
  'MAX': array([13500000.]),
  'MEAN': 1595757.9023},
 {'ACC': 98.87873667702982,
  'MAE': 115583.53,
  'MAX': array([13500000.]),
  'MEAN': 

In [24]:
res = nn.predict(X_test_scaled)

# y_test_wrong = y_scaled[:3483]

scaler_test = StandardScaler().fit(y_test.values)
res = scaler_test.inverse_transform(res)

mean_abs_error(res, y_test.values), max(y_test.values), round(np.mean(y_test.values), 2), round(r2_score(y_test.values, res), 4)

# scaler_test = StandardScaler().fit(y_test)
# res = scaler_test.inverse_transform(res)
# y_test_wrong = scaler_test.inverse_transform(y_test_wrong)
# print(res)

# mean_abs_error(res, y_test_wrong), max(y_test_wrong), round(np.mean(y_test_wrong), 2), round(r2_score(y_test_wrong, res), 4)

3483 3483 3483


(145589.35, array([13500000.]), 1688799.89, 0.9817)